# Algoritm

In [4]:
# Initiate environment

%matplotlib inline

import pandas as pd
#import DataFrame as df

import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import sys
reload(sys)
sys.setdefaultencoding('utf-8')


In [5]:
# Read the Data

# Todo, slice years

def read_data (csv_file,year, rows_to_read, dates_rows_to_parse):
    dateparser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
    df_data = pd.read_csv(csv_file,
                     delimiter= "\t", 
                     encoding= "utf-8",
                     nrows= rows_to_read, 
                     parse_dates = dates_rows_to_parse, 
                     infer_datetime_format = True,
                     date_parser=dateparser,
                     low_memory=False
                    )
    df_data = df_data[df_data['BDate'].dt.year == year_slice]
    # Add colunm for Cluster number
    df_data[cluster_field] = "0"
    df_data[cluster_distance_to_clusteroid_field] = np.nan
    return df_data

In [6]:
# N random clusteroids
def random_clusteroids (dataframe):
    clusteroids = dataframe.sample(number_of_clusteroids)[clusteroid_id_field].index.values
    print ("Random Clusteroids {}".format(clusteroids))
    return clusteroids

In [7]:
def haming_distance (row1, row2):
    unique_columns = row1 == row2
    lenght = len(columns_to_count_distance)
    unique_columns_sum = unique_columns.sum()
    distance = lenght - unique_columns_sum
    return distance
    

In [8]:
# Assign to clusters

# Todo: Add converging, isChanged

def assign_clusters (clusteroids_to_assign, dataframe):
    for point in dataframe.index.values: # Iterete through data to clusterise
        min_distance = len(columns_to_count_distance)
        min_clusteroid = -1
        for clusteroid in clusteroids_to_assign: # iterate throught clusters
            distance = haming_distance(dataframe.loc[point][columns_to_count_distance],
                              dataframe.loc[clusteroid][columns_to_count_distance])
            if distance < min_distance:
                min_distance = distance
                min_clusteroid = clusteroid #df_data.loc[clusteroid][clusteroid_id_field].index.values
        dataframe.set_value(point,cluster_field,min_clusteroid)
        dataframe.set_value(point,cluster_distance_to_clusteroid_field,min_distance)
        #print ("Point {} assignbed to {} with distance {}".format(point, min_clusteroid, min_distance))

In [9]:
# Recalculate clusteriods 2

# Get old clusteroid, for each element in the cluster calculate sum of distances to all the other points in cluster. 
# Keep Min -> save as new clusteroid

def recalculate_clusters (clusteroids, dataframe):
    old_clusteroids = list(clusteroids)
    #print ""
    print "---------"
    print "Clusters {}".format(clusteroids)
    old_clusteroids = np.copy(clusteroids)
    clusteroids = []
    for clusteroid in old_clusteroids: # Iterete through to clusteroids
        print ("Cluster {}".format(clusteroid))
        min_distance = dataframe[dataframe[cluster_field] == clusteroid][cluster_distance_to_clusteroid_field].sum()
        min_clusteroid = clusteroid
        for element_i in dataframe[dataframe[cluster_field] == clusteroid].index.values: # iterate throught cluster
            #print ("--Element_i {}".format(element_i))
            sum_distance = 0
            sum_clusteroid = clusteroid
            for element_j in dataframe[dataframe[cluster_field] == clusteroid].index.values: # iterate throught cluster
                distance = haming_distance(dataframe.loc[element_i][columns_to_count_distance],
                                           dataframe.loc[element_j][columns_to_count_distance])
                sum_distance = sum_distance + distance
            if sum_distance < min_distance:
                min_distance = sum_distance
                min_clusteroid = element_i
                print ("----New clusteroid {} min summ distance {}".format(element_i,sum_distance))
        clusteroids.append(min_clusteroid)
    print "Old Clusters {}".format(old_clusteroids)
    print "Clusters {}".format(clusteroids)
    print "---------"
    #print ""
    return clusteroids

In [10]:
# Constants

file_name = "uc_kep_obs/pays.txt"

# Number of clusters
number_of_clusteroids = 4

year_slice = 2016

num_of_rows = 500

# Columns to compare
columns_to_count_distance=[
    "region_client",
    "region_sc",
    "product_name",
    "product",
    "tarif_name",
    "price_item_name",
    "price_set_name",
    "month_length",
    "is_first",
    "Cnt",
]
dates_rows = [11,12]

# Clusteroid ID field
clusteroid_id_field = "id_org"
cluster_field = "Cluster"
cluster_distance_to_clusteroid_field = "Cluster_distance"


In [102]:
# program
print "---------"
print "Finde Clusteroids"
print ""

df_to_cluster = read_data(file_name, year_slice, num_of_rows, dates_rows)
new_clusteroids_array = random_clusteroids(df_to_cluster)
clusteroids_array = []
step = 1
while (clusteroids_array != new_clusteroids_array):
    clusteroids_array = list(new_clusteroids_array)
    print "---------"
    print "Run N {}".format(step)
    print ""
    assign_clusters(clusteroids_array, df_to_cluster)
    new_clusteroids_array = recalculate_clusters(clusteroids_array, df_to_cluster)
    step = step + 1
print ("Final clusteroids {}".format(new_clusteroids_array))

for cluster_1 in new_clusteroids_array:
    for cluster_2 in new_clusteroids_array:
        distance = haming_distance(df_to_cluster.loc[cluster_1][columns_to_count_distance],
                                   df_to_cluster.loc[cluster_2][columns_to_count_distance])
        print ("Distance from {} to {} is {}".format(cluster_1, cluster_2, distance))

In [61]:
# Save results
df_to_cluster.to_csv("temp3_last_clustering.csv",index=True, encoding="utf-8")

In [114]:
new_clusteroids_array

[143, 1, 22, 10]

In [125]:
pd.set_option('display.max_columns', None)
df_to_cluster.ix[new_clusteroids_array,:]

,id_org,region_client,id_number,vcuID,region_sc,product_name,product,tarif_name,price_item_name,price_set_name,month_length,BDate,PayDate,cost,tip,BaseCost,is_first,Cnt,amount,Cluster,Cluster_distance
143,D599C610-7452-41E7-9850-74018755CB19,1,1CE9CDC9-421A-4F2D-8584-6AE6CD24C206,C8F7BB9B-EE44-46A8-83C4-B88DE169F8CD,1,Сертум,Услуги УЦ,Организация рабочего места,Лицензия на право использования СКЗИ «КриптоПр...,Прайс Billy,NaN,2016-04-11,2016-04-12,1000.0,10,1000.0,1,1.0,830.7692,143,1.0
1,EB298F52-2EF5-47CA-B214-BF3021FBDE84,1,740BAECE-9FE6-45FF-8378-1890847685F3,C8F7BB9B-EE44-46A8-83C4-B88DE169F8CD,1,Сертум,Услуги УЦ,Сертум.Классик,Право использования программ для ЭВМ для управ...,Прайс Billy,12.0,2016-10-18,2016-10-21,4000.0,4,4000.0,1,1.0,4000.0000,1,0.0
22,7C930F72-9B3D-417E-902C-CF5ACAE407DC,1,A2081388-84D8-41BE-A5D3-05CA5E408A0E,98274BF4-5ACE-4C17-86A2-2B31AE03C07B,1,КЭП,КЭП,КЭП.Классик,"Абонентское обслуживание по тарифному плану ""К...",Прайс Billy,12.0,2016-03-22,2016-03-28,520.0,1,650.0,1,1.0,520.0000,22,0.0
10,85E1E31A-832A-4712-94D9-E7C919C217C7,1,22383A84-CAA2-44F6-B425-79A61F6FC0CF,420E16BF-3EBE-491A-997E-5B078E6CB3CC,23,КЭП для ЕГАИС,КЭП,КЭП.ЕГАИС на 12 мес.,"Абонентское обслуживание по тарифному плану ""К...",Прайс Billy,12.0,2016-06-02,2016-06-03,720.0,1,900.0,1,1.0,720.0000,10,0.0


In [88]:
#pd.DataFrame(squareform(pdist(df.ix[:, 1:])), columns=df.CITY.unique(), index=df.CITY.unique())
test_array = [(10,),(375,)]
from scipy.spatial.distance import pdist
pdist(test_array, lambda u, v: haming_distance(df_to_cluster.loc[u][columns_to_count_distance],
                                      df_to_cluster.loc[v][columns_to_count_distance]))

ValueError: Can only compare identically-labeled DataFrame objects

In [11]:
df_to_cluster = read_data(file_name, year_slice, num_of_rows, dates_rows)

In [12]:
df_to_cluster.groupby(["id_org"])["product_name"].agg(['count'])

,count
id_org,
06644FBF-B1B2-440D-8828-19043C1D2452,1
076F2FB1-26C0-4784-9EED-62F2501C5BC8,1
07846214-4287-4568-82FC-CB43B43FB463,3
0A62D07D-8127-4AC4-B67E-D149C5D06C0B,1
0CF2DCD7-070D-43FC-9491-A8E3CD2D2088,1
0F99483D-E397-46A0-8102-49CBB132F432,2
0FE0EC2A-9790-4FEF-BE4A-ED6098432E00,1
1025E67A-5262-4A74-90DF-19A2C18A3F79,2
118B57FA-654E-46C8-A80F-F5A1F89263FF,2
